In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
import os


In [6]:
load_dotenv("../.env")
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [7]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
    "You are a helpful travel assistant.\n\n"
    "Context:\n{context}"),
    ("human",
     "Task:\nAnswer the user's question using ONLY the provided context.\n"
     "If the answer is not present, say \"I don't have enough information.\"\n\n"
     "User Question:\n{question}")
])


In [8]:
base_models = {
    "Mistral": HuggingFaceEndpoint(
        repo_id="mistralai/Mistral-7B-Instruct-v0.2",
        huggingfacehub_api_token=token,
    ),
    "Llama3": HuggingFaceEndpoint(
        repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
        huggingfacehub_api_token=token,
    ),
    "Gemma": HuggingFaceEndpoint(
        repo_id="google/gemma-2-2b-it",
        huggingfacehub_api_token=token,
    )
}


In [9]:
models = {
    name: ChatHuggingFace(llm=llm)
    for name, llm in base_models.items()
}


In [10]:
chains = {name: prompt_template | model for name, model in models.items()}

for name, chain in chains.items():
    response = chain.invoke({
        "context": "Hotel A and Hotel B are in Paris with rating 4.5",
        "question": "Which hotels are in Paris?"
    })

    print(f"\n{name} response:")
    print(response.content)



Mistral response:
 I. Don't. Have. Enough. Information. About. Any. Hotels. Beyond. The. Two. You. Provided: Hotel A and Hotel B, both in Paris with a rating of 4.5.

Llama3 response:
Hotel A and Hotel B are in Paris.

Gemma response:
Hotel A and Hotel B are in Paris. 

